<a href="https://colab.research.google.com/github/katiko89/Educational-projects/blob/main/Python2%D0%9B%D0%A01%2C_%D0%9C%D0%B0%D1%80%D0%BA%D0%B8%D0%BD%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Программирование на языке Python. Часть 2.

Преподаватель: Горохов Олег Евгеньевич.

e-mail: gorokhov-oe@cs.msu.ru




# Лабораторная работа № 2. Разделение смеси распределений. Байессовский классификатор. Ядровые подходы.

В рамках данного задания вам нужно будет работать с уже известным набором данных о [пассажирах Титаника](https://www.kaggle.com/competitions/titanic/data).  

## Этап 1. EDA.



Скачаем данные:

In [ ]:
# Эту ячейку менять не нужно!

!wget -O "titanic.csv" "https://raw.githubusercontent.com/legas377/TestRepository/main/titanic.csv"

--2024-03-29 07:36:45--  https://raw.githubusercontent.com/legas377/TestRepository/main/titanic.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61194 (60K) [text/plain]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  59.76K  --.-KB/s    in 0.01s   

2024-03-29 07:36:45 (5.23 MB/s) - ‘titanic.csv’ saved [61194/61194]



Сделаем необходимые import-ы:

In [ ]:
import pandas as pd
import numpy as np

Сохраним загруженный файл в объект класса `DataFrame` и выведем первые его 10 строк:

In [ ]:
df = pd.read_csv('titanic.csv')
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Оставьте только колонки `Pclass`, `Sex`, `Age`, `Sibsp`, `Parch`, `Fare` и `Survived`:

In [ ]:
df=df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Survived']]

In [ ]:
df.isna().any()

Pclass      False
Sex         False
Age          True
SibSp       False
Parch       False
Fare        False
Survived    False
dtype: bool

In [ ]:
a=float(df['Age'].mode())
df['Age']=df['Age'].fillna(a)

Заполните пропуски во всех признаках, а также преобразуйте значения всех признаков к числам.

Для этого этапа можно пользоваться [конспектом семинара 5](https://colab.research.google.com/drive/1oq6dYmoEwMOyWbvS6IplbqwZJ9Sft0zi?usp=sharing) из прошлого семестра.

In [ ]:
df.isna().any()

Pclass      False
Sex         False
Age         False
SibSp       False
Parch       False
Fare        False
Survived    False
dtype: bool

In [ ]:
#проверим тип данных в каждом стобце
df.dtypes

Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Survived      int64
dtype: object

In [ ]:
#преобразуем данные стоблца Sex в числовые
def sex_int(sex):
  if sex=='male':
    return 0
  else:
    return 1

def sex_for_row(row):
  s=row['Sex']
  return sex_int(row['Sex'])

df['Sex']=df.apply(sex_for_row, axis=1)

In [ ]:
df.dtypes

Pclass        int64
Sex           int64
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Survived      int64
dtype: object

Видим, что теперь все признаки стали числовыми

## Этап 2. Разделение смеси распределений.

Теперь попробуем разделить распределение признака Fare на 3 группы с помощью [Gaussian Mixture Models](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html):

In [ ]:
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, homogeneity_score
from sklearn.cluster import KMeans

fare=np.array(df['Fare']).reshape(-1, 1)

gmm = GaussianMixture(n_components = 3)
gmm.fit(fare)

GaussianMixture(n_components=3)

Выведите стандартное отклонение и среднее значение для каждой группы:

In [ ]:
pred_class=gmm.predict(fare)+1
for k in range(3):
  std=df['Fare'][np.where(pred_class==k+1)[0]].std()
  mean=df['Fare'][np.where(pred_class==k+1)[0]].mean()
  print(f'{k+1} class: std={std}, mean={mean}')

1 class: std=22.759196815911036, mean=64.29641213872831
2 class: std=7.933910560319051, mean=13.752848088235295
3 class: std=99.81881725000065, mean=216.28245526315789


## Этап 3. Исследование разных алгоритмов инициализации.

Самостоятельно изучите [материал](https://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_init.html#sphx-glr-auto-examples-mixture-plot-gmm-init-py) про использовании разных способов начальной инициализации центров кластеров.

Далее попробуйте провести кластеризацию с помощью Gaussian Mixture Models и с разными способами инициализации. При этом вычислите метрики: Силуэт и [homogenity_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_score.html#sklearn.metrics.homogeneity_score) (на основании известного значения `Pclass`).

In [ ]:
import sklearn

def get_initial_means(X, init_params, r):
  gmm = GaussianMixture(
        n_components=3, init_params=init_params, tol=1e-9, max_iter=0, random_state=r
        ).fit(X)
  return gmm.means_

methods = ["kmeans", "random_from_data", "k-means++", "random"]

for n, method in enumerate(methods):
  r = np.random.RandomState(seed=42)
  ini = get_initial_means(fare, method, r)
  gmm_i = GaussianMixture(
        n_components=3, max_iter=2000, means_init=ini, tol=1e-9, random_state=r
    ).fit(fare)
  X_pred_i = gmm_i.predict(fare)
  print(f'{methods[n]}: homogenity_score = {homogeneity_score(df["Pclass"], X_pred_i)},' ,
        f'silhouette_score = {silhouette_score(df["Pclass"].array.reshape(-1,1), X_pred_i)}' )

kmeans: homogenity_score = 0.20162081944461283, silhouette_score = 0.29339697932989534
random_from_data: homogenity_score = 0.3971623073431511, silhouette_score = 0.22652646006068108
k-means++: homogenity_score = 0.19616110734187847, silhouette_score = 0.15591425038465254
random: homogenity_score = 0.39716230734315117, silhouette_score = 0.22652646006068108


Выберите лучший алгоритм инициализации на основании указанных метрик.

На основании метрик можем выбрать `random` или `random_from_data`. Выберем random.

Сравните средние значения и стандартные отклонения для каждого выделенного класса и для групп, которые могут быть получены путём группировки исходного набора данных по колонке `Pclass` (группировку можно посмотреть в [конспекте семинара](https://colab.research.google.com/drive/1oq6dYmoEwMOyWbvS6IplbqwZJ9Sft0zi?usp=sharing)).

In [ ]:
r = np.random.RandomState(seed=42)
ini = get_initial_means(fare, 'random', r)
gmm_rand = GaussianMixture(
      n_components=3, max_iter=2000, means_init=ini, tol=1e-9, random_state=r
    ).fit(fare)

pred_class=gmm_rand.predict(fare)+1
for k in range(3):
  std=df['Fare'][np.where(pred_class==k+1)[0]].std()
  mean=df['Fare'][np.where(pred_class==k+1)[0]].mean()
  print(f'{k+1} class: std={std}, mean={mean}')

1 class: std=0.3869177494305977, mean=7.7775041958041955
2 class: std=77.85677558392068, mean=104.41825029239766
3 class: std=9.444391064140262, mean=19.848069815668207


In [ ]:
first_class=df.query('Pclass==1')['Fare'].describe()
second_class=df.query('Pclass==2')['Fare'].describe()
third_class=df.query('Pclass==3')['Fare'].describe()

classes=[first_class, second_class, third_class]

for n, pclass in enumerate(classes):
  print(f'{n+1} class: std={pclass["std"]}, mean={pclass["mean"]}')

1 class: std=78.38037264672882, mean=84.1546875
2 class: std=13.417398756149343, mean=20.662183152173913
3 class: std=11.778141704387307, mean=13.675550101832993


## Этап 4. Классификация.

Разделите выборку с помощью `train_test_split`, при этом на тренировочную часть отведите 70%, на тестовую 30%. Установите `random_state=42`:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(df['Fare'], df['Pclass'], test_size=0.30, random_state=42)
X_train=X_train.array.reshape(-1,1)
X_test=X_test.array.reshape(-1,1)

Рассмотрите следующие алгоритмы классификации:

1. [Гауссовский наивный Байесс](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB).
2. [Метод опорных векторов](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).
3. Метод опорных векторов с разными ядровыми функциями.
4. (Дополнительно) Ансамбль указанных выше алгоритмов. Этот ансамбль строится по следующему принципу:
  * Состоит из трёх моделей, каждая модель решает задачу бинарной классификации для одного из кластеров, полученных в результате применения GMM;
  * Обучение каждого алгоритма: из тренировочной выборки выбираем только данные, принадлежащие одному кластеру, и обучаем соответствующую модель на них;
  * Применение: в зависимости от значения в колонке Pclass определяем, какую из моделей нужно применить.

При этом, для каждого алгоритма подберите оптимальные гиперпараметры на основании вычисления метрики `accuracy`. Укажите, какой алгоритм является лучшим с точки зрения `accuracy` и `F1-score`.

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score

gnb=GaussianNB()
gnb.fit(X_train, y_train)
prediction=gnb.predict(X_test)
acc=accuracy_score(y_test, prediction)
f1_sc=f1_score(y_test, prediction, average='macro')
print(f'accuracy: {acc}, \nF1-score: {f1_sc}')

accuracy: 0.6940298507462687, 
F1-score: 0.5084269662921348


In [ ]:
from sklearn.model_selection import GridSearchCV

svc=SVC()

pipe=Pipeline(steps=[('svc', svc)])
param_grid={
    "svc__kernel": ['linear', 'poly', 'rbf', 'sigmoid'],
    "svc__degree": [2,3,4],
    "svc__random_state": [42]
}

for metric in ['accuracy', 'f1_macro']:
  grid_search_pipe=GridSearchCV(pipe, param_grid, cv=None, scoring=metric)
  grid_search_pipe.fit(X_test, y_test)
  print(f'{metric}: {grid_search_pipe.best_score_}, \nСоответсвующие парамтеры: {grid_search_pipe.best_params_}')

accuracy: 0.7239692522711392, 
Соответсвующие парамтеры: {'svc__degree': 2, 'svc__kernel': 'linear', 'svc__random_state': 42}
f1_macro: 0.5557308402439961, 
Соответсвующие парамтеры: {'svc__degree': 2, 'svc__kernel': 'linear', 'svc__random_state': 42}
